# Create and Invoke Agent via Boto3 SDK

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

This notebook demonstrates the usage of the `bedrock-agent` and `bedrock-agent-runtime` boto3 clients to:
- Create an agent
- Create an action group
- Associate the agent with the action group and prepare it for use
- Create an agent alias
- Invoke the agent

We'll be utilizing Bedrock's Claude v3 Sonnet through the Boto3 API.

**Note:** *This notebook is designed to be run both within and outside of an AWS environment.*

### Prerequisites

Ensure you have an AWS account with permissions to:
- Create and manage IAM roles and policies
- Create and invoke AWS Lambda functions
- Create, read from, and write to Amazon S3 buckets
- Access and manage Amazon Bedrock agents and models
- Create and manage Amazon Glue databases and crawlers
- Execute queries and manage Amazon Athena workspaces

### Context

The following sections guide you through creating and invoking a Bedrock agent using the Boto3 SDK.

### Use Case

The notebook sets up an agent capable of crafting SQL queries from natural language questions. It then retrieves responses from the database, providing accurate answers to user inquiries. The diagram below outlines the high-level architecture of this solution.

![sequence-flow-agent](images/text-to-sql-architecture-Athena.png)

The Agent is designed to:
- Retrieve database schemas
- Execute SQL queries


In [1]:
upgrade_output = !pip install --upgrade pip
install_boto3_output = !pip install boto3
##Ensure your boto3 and botocore libraries are up to date
upgrade_output_botocore_boto3= !pip install --upgrade boto3 botocore 

In [75]:
from dependencies.config import *

In [76]:
!python ./dependencies/build_infra.py

[2024-10-27 21:00:30,494] p44054 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
TheHistoryOfBaseball
AccountID:  842676020002
arn:aws:iam::842676020002:user/bedrockfullaccess
AWSGlueServiceRole
s3://text-2-sql-agent-us-east-1-842676020002/data/TheHistoryofBaseball/
unzip_data()... finished
upload_data() ... finished
aws s3 sync ./data/extracted/ s3://text-2-sql-agent-us-east-1-842676020002/data
upload: data/extracted/TheHistoryofBaseball/hall_of_fame/hall_of_fame.csv to s3://text-2-sql-agent-us-east-1-842676020002/data/TheHistoryofBaseball/hall_of_fame/hall_of_fame.csv
upload: data/extracted/TheHistoryofBaseball/player_award_vote/player_award_vote.csv to s3://text-2-sql-agent-us-east-1-842676020002/data/TheHistoryofBaseball/player_award_vote/player_award_vote.csv
upload: data/extracted/TheHistoryofBaseball/salary/salary.csv to s3://text-2-sql-agent-us-east-1-842676020002/data/TheHistoryofBaseball/salary/salary.csv
upload: data/extracted/Th

In [46]:
from dependencies.config import *

# Create Agent
agent_instruction = """You are an expert database querying assistant that can create simple and complex SQL queries to get 
the answers to questions about baseball players that you are asked. You first need to get the schemas for the table in the database to then query the 
database tables using a sql statement then respond to the user with the answer to their question and
the sql statement used to answer the question. Use the getschema tool first to understand the schema
of the table then create a sql query to answer the users question.
Here is an example to query the table <example>SELECT * FROM thehistoryofbaseball.players LIMIT 10;</example> Do not use 
quotes for the table name. Your final answer should be in plain english."""


##PLEASE Note
###Disabling pre-processing can enhance the agent's response time, however, it may increase the risk of inaccuracies in SQL query generation or some sql ingestion. Careful consideration is advised when toggling this feature based on your use case requirements.


response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for performing sql queries.",
    idleSessionTTLInSeconds=idleSessionTTLInSeconds,
    foundationModel=foundation_Model,
    instruction=agent_instruction,
    promptOverrideConfiguration={
    #Disable preprocessing prompt
        'promptConfigurations': [
            {
                'promptType': 'PRE_PROCESSING',
                'promptCreationMode': 'OVERRIDDEN',
                'promptState': 'DISABLED',
                'basePromptTemplate':' ',
                 'inferenceConfiguration': {
                    'temperature': 0,
                    'topP': 1,
                    'topK': 123,
                    'maximumLength': 2048,
                    'stopSequences': [
                        'Human',
                    ]
                },
                
            }
        ]
    }
)
    


ValidationException: An error occurred (ValidationException) when calling the CreateAgent operation: Your account is not authorized to invoke this API operation for the provided model.

In [77]:


# Looking at the created agent, we can see its status and agent id






# Let's now store the agent id in a local variable to use it on the next steps




agent_id = "TISZBE0CSX"
agent_id


# ### Create Agent Action Group
# We will now create and agent action group that uses the lambda function and API schema files created before.
# The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.




# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': "arn:aws:lambda:us-east-1:842676020002:function:text-2-sql-agent-us-east-1-842676020002"
    },
    actionGroupName='QueryAthenaActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for getting the database schema and querying the Athena database'
)





agent_action_group_response


# ### Allowing Agent to invoke Action Group Lambda
# Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created




# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)


# ### Preparing Agent
# Let's create a DRAFT version of the agent that can be used for internal testing.




agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare


# ### Create Agent alias
# We will now create an alias of the agent that can be used to deploy the agent.




# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)




# Pause to make sure agent alias is ready
time.sleep(30)

agent_alias

print(agent_alias)


{'ResponseMetadata': {'RequestId': 'b404234e-a6d2-4b2f-aa91-0c2849cf75cc', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Sun, 27 Oct 2024 21:09:32 GMT', 'content-type': 'application/json', 'content-length': '340', 'connection': 'keep-alive', 'x-amzn-requestid': 'b404234e-a6d2-4b2f-aa91-0c2849cf75cc', 'x-amz-apigw-id': 'AU_R-H55IAMEhlA=', 'x-amzn-trace-id': 'Root=1-671eac0c-45a807b21a6a55e335ba3646'}, 'RetryAttempts': 0}, 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-east-1:842676020002:agent-alias/TISZBE0CSX/TBM0MVRUDC', 'agentAliasId': 'TBM0MVRUDC', 'agentAliasName': 'workshop-alias', 'agentAliasStatus': 'CREATING', 'agentId': 'TISZBE0CSX', 'createdAt': datetime.datetime(2024, 10, 27, 21, 9, 32, 314360, tzinfo=tzutc()), 'routingConfiguration': [{}], 'updatedAt': datetime.datetime(2024, 10, 27, 21, 9, 32, 314360, tzinfo=tzutc())}}


In [78]:
from dependencies.config import *

list_agent=bedrock_agent_client.list_agents()['agentSummaries']
#print(list_agent)
#print(agent_name)
# Search f
agent_id = next((agent['agentId'] for agent in list_agent if agent['agentName'] == agent_name), None)

print(agent_id)

response = bedrock_agent_client.list_agent_aliases(
    agentId=agent_id,
)
response['agentAliasSummaries']
agentAliasId=next((agent['agentAliasId'] for agent in response['agentAliasSummaries'] if agent['agentAliasName'] == agent_alias_name), None)
agent_alias_id=agentAliasId
print(agent_alias_id)

TISZBE0CSX
TBM0MVRUDC


In [79]:
# ### Invoke Agent
# Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.
query_to_agent = """What was John Denny's salary in 1986?"""

## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = True
end_session:bool = False


# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText=query_to_agent,
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)
logger.info(pprint.pprint(agentResponse))

[2024-10-27 21:10:18,811] p88226 {2468800596.py:20} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sun, 27 Oct 2024 21:10:18 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'ddc543dc-94a7-11ef-8e96-663f7c7041cd',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '10ab9a72-e2df-4cd2-b708-7643dea4a0dc'},
                      'HTTPStatusCode': 200,
                      'RequestId': '10ab9a72-e2df-4cd2-b708-7643dea4a0dc',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x11d612e90>,
 'contentType': 'application/json',
 'sessionId': 'ddc543dc-94a7-11ef-8e96-663f7c7041cd'}


In [80]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-10-27 21:10:19,121] p88226 {<timed exec>:11} INFO - {
  "agentAliasId": "TBM0MVRUDC",
  "agentId": "TISZBE0CSX",
  "agentVersion": "1",
  "sessionId": "ddc543dc-94a7-11ef-8e96-663f7c7041cd",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [],
          "temperature": 0.0,
          "topK": 1,
          "topP": 1.000000013351432e-10
        },
        "text": "System: A chat between a curious User and an artificial intelligence Bot. The Bot gives helpful, detailed, and polite answers to the User's questions. In this session, the model has access to external functionalities.\nTo assist the user, you can reply to the user or invoke an action. Only invoke actions if relevant to the user request.\nYou are an expert database querying assistant that can create simple and complex SQL queries to get \nthe answers to questions about baseball players that you are asked. Y

Exception: ('unexpected event.', EventStreamError('An error occurred (accessDeniedException) when calling the InvokeAgent operation: Access denied when calling Bedrock. Check your request permissions and retry the request.'))

In [59]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

NameError: name 'agent_answer' is not defined

In [ ]:
#Create function to invoke agent
def invoke_agent(query):
    ## create a random id for session initiator id
    session_id:str = str(uuid.uuid1())
    enable_trace:bool = False
    end_session:bool = False
 

    # invoke the agent API
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    event_stream = agentResponse['completion']
    print("Fetching answer...")
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)



In [ ]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")

In [ ]:
invoke_agent("What year was Nolan Ryan inducted into the Hall of Fame?")


In [ ]:
invoke_agent("In what year did Hank Aaron hit the most home runs?")


In [ ]:
#This query requires a join of two tables to be able to answer the question
invoke_agent("What player has received the most All-Star Game selections?")

In [ ]:
#This query should say there is no data available for this year!!
invoke_agent("What was Babe Ruth's salary in 1930?")


In [ ]:
invoke_agent("Who is the richest player in baseball history? ")


In [ ]:
invoke_agent("What was John Denny's salary in 1986? ")


In [ ]:
# This will stop the notebook's execution and wait for the user to press Enter
input("Press Enter to continue...")


## Clean up (optional)

The next steps are optional and delete the infrustcture we built. 



In [81]:
!python ./dependencies/clean.py

[2024-10-27 21:38:28,931] p51014 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
TheHistoryOfBaseball
Crawler 'TheHistoryOfBaseball' deleted successfully.
Table 'hall_of_fame' deleted successfully.
Table 'player' deleted successfully.
Table 'player_award' deleted successfully.
Table 'player_award_vote' deleted successfully.
Table 'salary' deleted successfully.
Database 'thehistoryofbaseball' deleted successfully.
TISZBE0CSX
[{'actionGroupId': '6LWYFGJUZV', 'actionGroupName': 'QueryAthenaActionGroup', 'actionGroupState': 'ENABLED', 'description': 'Actions for getting the database schema and querying the Athena database', 'updatedAt': datetime.datetime(2024, 10, 27, 21, 9, 33, 424151, tzinfo=tzutc())}]
6LWYFGJUZV
<class 'list'>
text-2-sql-agent-us-east-1-842676020002
arn:aws:lambda:us-east-1:842676020002:function:text-2-sql-agent-us-east-1-842676020002
can not delete
Policy 'text-2-sql-agent-allow-us-east-1-842676020002' deleted successfully.

In [63]:
# Use the Converse API to send a text message to Titan Text G1 - Express.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Titan Text Premier.
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Start a conversation with the user message.
user_message = """Meeting transcript: 
Miguel: Hi Brant, I want to discuss the workstream  for our new product launch 
Brant: Sure Miguel, is there anything in particular you want to discuss? 
Miguel: Yes, I want to talk about how users enter into the product. 
Brant: Ok, in that case let me add in Namita. 
Namita: Hey everyone 
Brant: Hi Namita, Miguel wants to discuss how users enter into the product. 
Miguel: its too complicated and we should remove friction.  for example, why do I need to fill out additional forms?  I also find it difficult to find where to access the product when I first land on the landing page. 
Brant: I would also add that I think there are too many steps. 
Namita: Ok, I can work on the landing page to make the product more discoverable but brant can you work on the additonal forms? 
Brant: Yes but I would need to work with James from another team as he needs to unblock the sign up workflow.  Miguel can you document any other concerns so that I can discuss with James only once? 
Miguel: Sure. 
From the meeting transcript above, Create a list of action items for each person. 
"""
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId="amazon.titan-text-express-v1",
        messages=conversation,
        inferenceConfig={"maxTokens":4096,"stopSequences":["User:"],"temperature":0,"topP":1},
        additionalModelRequestFields={}
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)


ERROR: Can't invoke 'anthropic.claude-3-sonnet-20240229-v1:0'. Reason: An error occurred (AccessDeniedException) when calling the Converse operation: You don't have access to the model with the specified model ID.



## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You

In [65]:
import json
import pandas as pd
from datetime import datetime, timedelta

class DataAnalyticsApp:
    def __init__(self):
        # Initialize AWS clients
        self.bedrock = boto3.client('bedrock-runtime')
        self.athena = boto3.client('athena')
        
    def generate_sql(self, question: str) -> str:
        """Generate SQL using Amazon Titan (most cost-effective option)"""
        prompt = f"""You are an expert SQL writer. Convert this question to SQL.
        Table: safe_marketing_metrics (
            event_date DATE,
            platform STRING,
            campaign_id STRING,
            unique_users INTEGER,
            event_count INTEGER
        )
        Question: {question}
        Write only the SQL query, no explanations."""
        
        body = json.dumps({
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 300,
                "temperature": 0.1,
                "topP": 0.9
            }
        })
        
        response = self.bedrock.invoke_model(
            modelId='amazon.titan-text-express-v1',
            body=body
        )
        return json.loads(response['body'].read())['results'][0]['outputText']

    def run_query(self, sql: str) -> pd.DataFrame:
        """Execute Athena query"""
        response = self.athena.start_query_execution(
            QueryString=sql,
            ResultConfiguration={
                'OutputLocation': 's3://athena-query-results-842676020002/query-results/'
            }
        )
        
        # Wait for query completion
        query_id = response['QueryExecutionId']
        while True:
            status = self.athena.get_query_execution(QueryExecutionId=query_id)
            state = status['QueryExecution']['Status']['State']
            if state in ['SUCCEEDED', 'FAILED', 'CANCELLED']:
                break
            time.sleep(1)
            
        if state == 'SUCCEEDED':
            # Get results
            results = self.athena.get_query_results(QueryExecutionId=query_id)
            # Convert to DataFrame
            columns = [col['Name'] for col in results['ResultSet']['ResultSetMetadata']['ColumnInfo']]
            data = []
            for row in results['ResultSet']['Rows'][1:]:  # Skip header
                data.append([field.get('VarCharValue', '') for field in row['Data']])
            return pd.DataFrame(data, columns=columns)
        else:
            raise Exception(f"Query failed with state: {state}")


[2024-10-27 18:56:59,163] p88226 {utils.py:148} INFO - Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2024-10-27 18:56:59,164] p88226 {utils.py:160} INFO - NumExpr defaulting to 8 threads.


In [69]:
import streamlit as st
# Initialize app state
if 'analytics_app' not in st.session_state:
    st.session_state.analytics_app = DataAnalyticsApp()

2024-10-27 19:02:37.594 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-27 19:02:37.594 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2024-10-27 19:02:37.632 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-27 19:02:37.633 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [67]:
# Use the Converse API to send a text message to Titan Text G1 - Premier.

import boto3
from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region you want to use.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Titan Text Premier.
model_id = "amazon.titan-text-premier-v1:0"

# Start a conversation with the user message.
user_message = """You are an expert Business Analyst at a fitness company with more than 10,000 daily active users on the app and website.

You are tasked to do a market research in comparing multiple brands of shoes for a potential partnership.

The goal for these shoes are to be used for workouts, so they should be easy fit, good for running, walking and jumping, and be affordable for your customers. After this partnership, customers can buy the partners shoes from your website at a discounted price and create a win-win opportunity for the partner as well as for your business and customers.

Tha main objective when doing this analysis is 
- Customer Satisfaction
- Inclusion for all ages, sizes, and purchasing power
- Brand visibiliy and partnership

Help me with a step-by-step plan on how to go about doing this analysis and what research I should do to come to a conclusion.

Guide me and help me think step-by-step. Explain your thought process as you work through this process. 
"""
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId="amazon.titan-text-premier-v1:0",
        messages=conversation,
        inferenceConfig={"maxTokens":1024,"stopSequences":[],"temperature":0.7,"topP":0.9},
        additionalModelRequestFields={}
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)


Step 1: Compile a list of potential shoe brands

Start by compiling a list of potential shoe brands that cater to workouts, are easy to fit, suitable for running, walking, and jumping, and are affordable. You can gather this information through online research, industry reports, and networking with other professionals in the fitness industry.

Step 2: Analyze the market for each brand

Once you have a list of potential partners, analyze the market for each brand. Look at their market share, customer base, pricing, and distribution channels. This information will help you understand the competitive landscape and identify potential opportunities for partnership.

Step 3: Evaluate customer satisfaction

Next, evaluate customer satisfaction for each brand. Look at customer reviews, ratings, and feedback. You can also conduct surveys or focus groups to gather more in-depth information about customer satisfaction.

Step 4: Assess brand visibility and partnership opportunities

Assess the bra